# Exercise: Range Minimum Queries (RMQ)

This exercise explores six different approaches to solving the ***Range Minimum Queries (RMQ)*** problem, analyzing their performance based on the following metrics::

* ***Querying and Preprocessing Completed Time (Run Time):*** Measures the total time taken to preprocess the data and execute queries.
* ***Memory Usage During Preprocessing and Querying:*** Tracks the current memory usage and peak memory usage during both preprocessing and query operations.
* ***Time Complexity of Preprocessing and Querying:*** Analyzes the theoretical time complexity of the algorithms involved in preprocessing and querying.

The implemented approaches include:
* ***Naive:*** A brute-force approach that iterates through the entire subarray to find the minimum.
* ***Precomputation-based:*** Precomputes the minimum values for all possible subarrays, allowing for constant-time queries.
* ***Block Decomposition:*** Divides the array into blocks and precomputes the minimum for each block, improving query efficiency for large arrays.
* ***Sparse Table:*** A data structure that uses dynamic programming to efficiently compute and store minimum values for subarrays of various sizes.
* ***Hybrid Solutions*** (Two approaches): Combine elements of block decomposition and sparse table to achieve a balance of preprocessing time and query efficiency.


All approaches are implemented on the same initial **array with a size of 2^16**, and their performance is evaluated on both this array and a larger array with **a size of 2^17**. Each approach is implemented in a separate code cell for clarity and comparison.


In [1]:
import math
import random
import time
import sys
import pickle  # For saving the precomputed RMQ table
import tracemalloc  # For measuring memory usage
import numpy as np

In [2]:
# Generate an array of length 2^x with random integers
x = 5
n = 2**x
A = [random.randint(1, 100000) for _ in range(n)]  # array with random values
print(A)

# Predefined i and j values
i = 10
j = 30

[17149, 51817, 55033, 28741, 29854, 2419, 68238, 45906, 23933, 70293, 33476, 98082, 64471, 29387, 52905, 83494, 38390, 82377, 70781, 8048, 34361, 23764, 1411, 67169, 79248, 84840, 82170, 85250, 10403, 25527, 49499, 85139]


# Approach 1: Naive

In [3]:
def RMQ_naive(A, i, j):
    min_value = A[i]
    comparisons = 0  # Track number of comparisons
    start_time = time.time()  # Start time measurement
    for k in range(i + 1, j + 1):
        comparisons += 1
        if A[k] < min_value:
            min_value = A[k]
    end_time = time.time()  # End time measurement
    total_time = end_time - start_time  # Calculate time taken
    entries_evaluated = j - i + 1  # Number of entries in the range
    avg_time_per_entry = total_time / entries_evaluated  # Average time per entry
    return min_value, entries_evaluated, avg_time_per_entry, total_time, comparisons


# Measuring Naive
def naive_evaluation():
    tracemalloc.start()
    snapshot1 = tracemalloc.take_snapshot()
    
    # Ensure that i and j are within valid range
    if 0 <= i <= j < n:
        # Call the RMQ function
        min_val, entries, avg_time, total_time, comparisons = RMQ_naive(A, i, j)
        
        # Memory snapshot after RMQ
        snapshot2 = tracemalloc.take_snapshot()
        top_stats = snapshot2.compare_to(snapshot1, 'lineno')
        
        # Memory usage stats
        mem_usage = sum(stat.size_diff for stat in top_stats)
        
        # Print results
        print(f"The minimum value in the range [{i}, {j}] is: {min_val}")
        print(f"Number of entries evaluated: {entries}")
        print(f"Average time to evaluate each entry: {avg_time:.10f} seconds")
        print(f"Total time required: {total_time:.10f} seconds")
        print(f"Number of comparisons made: {comparisons}")
        print(f"Memory usage during RMQ: {mem_usage / 1024:.2f} KB")
        print(f"Time complexity: O(n) where n = {entries}")
    else:
        print("Invalid input. Please ensure that 0 <= i <= j < n.")
    
    tracemalloc.stop()

# Measure memory usage and run the analysis
naive_evaluation()

The minimum value in the range [10, 30] is: 1411
Number of entries evaluated: 21
Average time to evaluate each entry: 0.0000000000 seconds
Total time required: 0.0000000000 seconds
Number of comparisons made: 20
Memory usage during RMQ: 0.62 KB
Time complexity: O(n) where n = 21


# Approach 2: Precomputation-Based

In [4]:
# Precompute all RMQ values using a bottom-up dynamic programming approach with linear scanning
def precompute_RMQ(A):
    n = len(A)
    M = [[float('inf')] * n for _ in range(n)]  # Initialize the precomputed table with infinity
    
    num_entries_evaluated = 0  # Track how many entries are evaluated
    num_comparisons = 0  # Track how many comparisons are made
    
    start_time = time.time()  # Start timing the precomputation process

    # Fill the table for subarrays of length 1
    for i in range(n):
        M[i][i] = A[i]  # Minimum for subarray [i, i] is A[i]
        num_entries_evaluated += 1  # We evaluate one entry per subarray of length 1
    
    # Fill the table for subarrays of length >= 2
    for length in range(2, n + 1):  # length ranges from 2 to n
        for i in range(n - length + 1):
            j = i + length - 1  # Calculate the end index
            # Compute the minimum for subarray [i, j]
            M[i][j] = min(M[i][j - 1], A[j])
            num_entries_evaluated += 1  # One more entry evaluated
            num_comparisons += 1  # One comparison made per entry
    
    end_time = time.time()  # End timing the precomputation process

    total_time = end_time - start_time
    avg_time_per_entry = total_time / num_entries_evaluated

    # Calculate memory usage for the precomputed table
    memory_usage = sys.getsizeof(M) + sum(sys.getsizeof(row) for row in M)

    print(f"Analysis of RMQ Precomputation:")
    print(f"Number of entries evaluated: {num_entries_evaluated}")
    print(f"Average time to evaluate each entry: {avg_time_per_entry:.6f} seconds")
    print(f"Total time required: {total_time:.6f} seconds")
    print(f"Number of comparisons made: {num_comparisons}")
    print(f"Memory usage for precomputed table: {memory_usage / (1024 * 1024):.2f} MB")
    print(f"Time taken to create the precomputed table: {total_time:.6f} seconds")  # Printing creation time
    
    # Save the precomputed table to a file
    #with open('rmq_table_x16.pkl', 'wb') as f:
    #    pickle.dump(M, f)
    #print("RMQ table has been saved as 'rmq_table.pkl'")

    return M, total_time, num_comparisons, memory_usage


# Query the precomputed table and measure memory usage
def RMQ_query(M, i, j):
    tracemalloc.start()  # Start tracking memory usage before the query
    start_query_time = time.time()  # Start timing the query process

    # Perform the RMQ query
    result = M[i][j]

    end_query_time = time.time()  # End timing the query process
    query_time = end_query_time - start_query_time
    
    current, peak = tracemalloc.get_traced_memory()  # Get current and peak memory usage
    tracemalloc.stop()  # Stop tracking memory

    # Return the result, query time, and memory usage during the query
    print(f"Query completed in {query_time:.6f} seconds.")
    print(f"Memory usage during query: Current: {current / 1024:.2f} KB, Peak: {peak / 1024:.2f} KB")
    return result, query_time, current, peak


#Precompute the RMQ table
M, precomp_time, num_comparisons, memory_usage = precompute_RMQ(A)

#with open('rmq_table_x17.pkl', 'rb') as f: ### rmq_table_x16
#    M = pickle.load(f)


# perform the query
def main():
    # Ensure that i and j are within valid range
    if 0 <= i <= j < n:
        # Perform the query in O(1) time and get memory usage
        min_val, query_time, current_mem, peak_mem = RMQ_query(M, i, j)
        print(f"The minimum value in the range [{i}, {j}] is: {min_val}")

        # Print final analysis of precomputation and query
        print(f"\nFinal Analysis:")
        print(f"Precomputation time: {precomp_time:.6f} seconds")
        print(f"Memory usage during RMQ precomputation: {memory_usage / (1024 * 1024):.2f} MB")
        print(f"Number of comparisons made during precomputation: {num_comparisons}")
        print(f"Query time: {query_time:.6f} seconds")
        print(f"Memory usage during query: Current: {current_mem / 1024:.2f} KB, Peak: {peak_mem / 1024:.2f} KB")
        print(f"Time complexity: O(n^2) for preprocessing, O(1) for queries")
    else:
        print("Invalid input. Please ensure that 0 <= i <= j < n.")



if __name__ == "__main__":
    main()

Analysis of RMQ Precomputation:
Number of entries evaluated: 528
Average time to evaluate each entry: 0.000000 seconds
Total time required: 0.000000 seconds
Number of comparisons made: 496
Memory usage for precomputed table: 0.01 MB
Time taken to create the precomputed table: 0.000000 seconds
Query completed in 0.000000 seconds.
Memory usage during query: Current: 0.00 KB, Peak: 0.00 KB
The minimum value in the range [10, 30] is: 1411

Final Analysis:
Precomputation time: 0.000000 seconds
Memory usage during RMQ precomputation: 0.01 MB
Number of comparisons made during precomputation: 496
Query time: 0.000000 seconds
Memory usage during query: Current: 0.00 KB, Peak: 0.00 KB
Time complexity: O(n^2) for preprocessing, O(1) for queries


# Approach 3: Block Decomposition

In [5]:
# Precompute block-level minimums and overall block minimums
def block_decomposition_RMQ(A):
    n = len(A)
    block_size = int(math.sqrt(n))
    num_blocks = math.ceil(n / block_size)
    
    # Array to store minimum of each block
    block_min = [float('inf')] * num_blocks
    
    # Start tracking memory usage for preprocessing
    tracemalloc.start()
    start_preprocess_time = time.time()

    # Precompute block minimums
    for i in range(n):
        block_index = i // block_size
        block_min[block_index] = min(block_min[block_index], A[i])

    # End timing and memory tracking
    end_preprocess_time = time.time()
    preprocess_time = end_preprocess_time - start_preprocess_time
    current, peak = tracemalloc.get_traced_memory()  # Get current and peak memory usage
    tracemalloc.stop()

    # Print the results of preprocessing
    print(f"Preprocessing completed in {preprocess_time:.6f} seconds.")
    print(f"Memory usage during preprocessing:")
    print(f"  Current memory usage: {current / 1024:.2f} KB")
    print(f"  Peak memory usage: {peak / 1024:.2f} KB")
    
    return block_min, block_size, num_blocks, preprocess_time, current, peak


# Querying function using block decomposition
def RMQ_block_query(A, block_min, block_size, i, j):
    min_val = float('inf')
    comparisons = 0
    
    # Process elements before the first fully covered block
    while i < j and i % block_size != 0:
        min_val = min(min_val, A[i])
        comparisons += 1
        i += 1

    # Process the fully covered blocks
    while i + block_size <= j:
        block_index = i // block_size
        min_val = min(min_val, block_min[block_index])
        comparisons += 1
        i += block_size

    # Process elements after the last fully covered block
    while i <= j:
        min_val = min(min_val, A[i])
        comparisons += 1
        i += 1

    return min_val, comparisons


# Query the block decomposition RMQ
def block_decomposition_query(A, block_min, block_size, i, j):
    tracemalloc.start()  # Start tracking memory usage
    start_query_time = time.time()  # Start timing the query process
    
    # Perform the RMQ query using block decomposition
    result, comparisons = RMQ_block_query(A, block_min, block_size, i, j)
    
    end_query_time = time.time()  # End timing the query process
    query_time = end_query_time - start_query_time

    current, peak = tracemalloc.get_traced_memory()  # Get current and peak memory usage
    tracemalloc.stop()  # Stop tracking memory

    # Return the result, query time, and memory usage during the query
    print(f"Query completed in {query_time:.6f} seconds.")
    print(f"Memory usage during query: Current: {current / 1024:.2f} KB, Peak: {peak / 1024:.2f} KB")
    print(f"Number of comparisons during query: {comparisons}")
    return result, query_time, comparisons, current, peak


# Precompute block decomposition with memory tracking
block_min, block_size, num_blocks, preprocess_time, preprocess_current_mem, preprocess_peak_mem = block_decomposition_RMQ(A)


#### perform the query
def main():

    # Ensure that i and j are within valid range
    if 0 <= i <= j < n:
        # Perform the query in O(sqrt(n)) time and get memory usage
        min_val, query_time, comparisons, query_current_mem, query_peak_mem = block_decomposition_query(A, block_min, block_size, i, j)
        print(f"The minimum value in the range [{i}, {j}] is: {min_val}")

        # Final analysis of preprocessing and query
        print(f"\nFinal Analysis Results:")
        print(f"Preprocessing completed in {preprocess_time:.6f} seconds.")
        print(f"Memory usage during preprocessing:")
        print(f"  Current memory usage: {preprocess_current_mem:.2f} KB")
        print(f"  Peak memory usage: {preprocess_peak_mem:.2f} KB")
        print(f"Time complexity of preprocessing: O(n)")
        
        print(f"\nQuery completed in {query_time:.6f} seconds.")
        print(f"Number of comparisons: {comparisons}")
        print(f"Memory usage during query:")
        print(f"  Current memory usage: {query_current_mem:.2f} KB")
        print(f"  Peak memory usage: {query_peak_mem:.2f} KB")
        print(f"Time complexity of query: O(sqrt(n))")
    else:
        print("Invalid input. Please ensure that 0 <= i <= j < n.")




if __name__ == "__main__":
    main()

Preprocessing completed in 0.000000 seconds.
Memory usage during preprocessing:
  Current memory usage: 0.00 KB
  Peak memory usage: 0.08 KB
Query completed in 0.000000 seconds.
Memory usage during query: Current: 0.00 KB, Peak: 0.05 KB
Number of comparisons during query: 5
The minimum value in the range [10, 30] is: 1411

Final Analysis Results:
Preprocessing completed in 0.000000 seconds.
Memory usage during preprocessing:
  Current memory usage: 0.00 KB
  Peak memory usage: 80.00 KB
Time complexity of preprocessing: O(n)

Query completed in 0.000000 seconds.
Number of comparisons: 5
Memory usage during query:
  Current memory usage: 0.00 KB
  Peak memory usage: 48.00 KB
Time complexity of query: O(sqrt(n))


# Approach 4: Sparse Table

In [6]:
# Function to build the sparse table
def build_sparse_table(A):
    n = len(A)
    log = math.floor(math.log2(n)) + 1  # The number of levels in the sparse table
    M = [[0] * log for _ in range(n)]  # Initialize the table with size n x log(n)
    
    comparisons = 0  # Track the number of comparisons

    # Start tracking memory usage for preprocessing
    tracemalloc.start()
    start_preprocess_time = time.time()

    # Initialize the table for intervals of length 1
    for i in range(n):
        M[i][0] = A[i]

    # Fill the sparse table for intervals of length 2^j
    j = 1
    while (1 << j) <= n:
        i = 0
        while (i + (1 << j) - 1) < n:
            M[i][j] = min(M[i][j - 1], M[i + (1 << (j - 1))][j - 1])
            comparisons += 1  # Increment the comparison counter
            i += 1
        j += 1

    # End timing and memory tracking
    end_preprocess_time = time.time()
    preprocess_time = end_preprocess_time - start_preprocess_time
    current, peak = tracemalloc.get_traced_memory()  # Get current and peak memory usage
    tracemalloc.stop()

    return M, preprocess_time, current / 1024, peak / 1024, comparisons


# Function to perform a range minimum query using the sparse table
def rmq_sparse_table(M, L, R):
    length = R - L + 1
    k = math.floor(math.log2(length))
    # Perform one comparison between two precomputed ranges
    return min(M[L][k], M[R - (1 << k) + 1][k]), 1  # Returning 1 comparison in the query phase


# Build the sparse table with memory tracking
M, preprocess_time, preprocess_current_mem, preprocess_peak_mem, preprocess_comparisons = build_sparse_table(A)


# perform the query
def main():

    # Ensure that i and j are within valid range
    if 0 <= i <= j < n:
        # Start timing and memory tracking for the query
        tracemalloc.start()
        start_query_time = time.time()

        # Perform the range minimum query
        min_val, query_comparisons = rmq_sparse_table(M, i, j)

        end_query_time = time.time()
        query_time = end_query_time - start_query_time
        current_query_mem, peak_query_mem = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        # Output the result and analysis
        print(f"The minimum value in the range [{i}, {j}] is: {min_val}")

        print(f"\nFinal Analysis Results:")
        print(f"Preprocessing completed in {preprocess_time:.6f} seconds.")
        print(f"Number of comparisons during preprocessing: {preprocess_comparisons}")
        print(f"Memory usage during preprocessing:")
        print(f"  Current memory usage: {preprocess_current_mem:.2f} KB")
        print(f"  Peak memory usage: {preprocess_peak_mem:.2f} KB")
        print(f"Time complexity of preprocessing: O(n log n)")

        print(f"\nQuery completed in {query_time:.6f} seconds.")
        print(f"Number of comparisons during query: {query_comparisons}")
        print(f"Memory usage during query:")
        print(f"  Current memory usage: {current_query_mem / 1024:.2f} KB")
        print(f"  Peak memory usage: {peak_query_mem / 1024:.2f} KB")
        print(f"Time complexity of query: O(1)")
    else:
        print("Invalid input. Please ensure that 0 <= i <= j < n.")



if __name__ == "__main__":
    main()

The minimum value in the range [10, 30] is: 1411

Final Analysis Results:
Preprocessing completed in 0.000998 seconds.
Number of comparisons during preprocessing: 103
Memory usage during preprocessing:
  Current memory usage: 0.00 KB
  Peak memory usage: 0.08 KB
Time complexity of preprocessing: O(n log n)

Query completed in 0.000000 seconds.
Number of comparisons during query: 1
Memory usage during query:
  Current memory usage: 0.00 KB
  Peak memory usage: 0.05 KB
Time complexity of query: O(1)


# Approach 5: Block Decomposition with Sparse Table

In [7]:
# Function to build RMQ for individual blocks (naive O(b) approach)
def build_block_rmq(block):
    n = len(block)
    M = [[0] * n for _ in range(n)]  # Initialize the block RMQ table
    comparisons = 0  # Track comparisons

    for i in range(n):
        M[i][i] = block[i]  # Initialize the diagonal (range of size 1)
        for j in range(i + 1, n):
            comparisons += 1
            M[i][j] = min(M[i][j - 1], block[j])
    
    return M, comparisons


# Function to perform RMQ within a block using the precomputed table
def rmq_block(M, L, R):
    return M[L][R]


# Function to build the macro RMQ structure on block minimums using sparse table
def build_sparse_table(block_mins):
    n = len(block_mins)
    log = math.floor(math.log2(n)) + 1  # The number of levels in the sparse table
    M = [[0] * log for _ in range(n)]  # Initialize the table with size n x log(n)]
    comparisons = 0  # Track comparisons

    # Initialize the table for intervals of length 1
    for i in range(n):
        M[i][0] = block_mins[i]

    # Fill the sparse table for intervals of length 2^j
    j = 1
    while (1 << j) <= n:
        i = 0
        while (i + (1 << j) - 1) < n:
            comparisons += 1
            M[i][j] = min(M[i][j - 1], M[i + (1 << (j - 1))][j - 1])
            i += 1
        j += 1
    
    return M, comparisons


# Function to perform RMQ on the block minimums using the sparse table
def rmq_sparse_table(M, L, R):
    length = R - L + 1
    k = math.floor(math.log2(length))
    return min(M[L][k], M[R - (1 << k) + 1][k])


# Hybrid RMQ that combines block decomposition and sparse table on block mins
def hybrid_rmq(A, block_size):
    n = len(A)
    num_blocks = math.ceil(n / block_size)

    # Step 1: Precompute block RMQs and block minimums
    blocks = []
    block_mins = []
    total_comparisons = 0

    for k in range(0, n, block_size):
        block = A[k:min(k + block_size, n)]
        block_rmq, comparisons = build_block_rmq(block)
        blocks.append(block_rmq)
        block_mins.append(min(block))  # Store the minimum of the block
        total_comparisons += comparisons

    # Step 2: Build sparse table for the block minimums
    sparse_table, comparisons = build_sparse_table(block_mins)
    total_comparisons += comparisons

    return blocks, sparse_table, total_comparisons


# Function to perform the hybrid RMQ query
def query_hybrid_rmq(A, blocks, sparse_table, block_size, i, j):
    left_min = None
    right_min = None
    middle_min = None

    start_block = i // block_size
    end_block = j // block_size

    if start_block == end_block:
        # Case 1: i and j are in the same block
        return rmq_block(blocks[start_block], i % block_size, j % block_size)

    # Case 2: i and j span multiple blocks
    left_min = rmq_block(blocks[start_block], i % block_size, block_size - 1)
    right_min = rmq_block(blocks[end_block], 0, j % block_size)

    if start_block + 1 <= end_block - 1:
        middle_min = rmq_sparse_table(sparse_table, start_block + 1, end_block - 1)
    else:
        middle_min = float('inf')

    return min(left_min, right_min, middle_min)


# Main code to execute the RMQ
def main():

    # Choose a block size
    block_size = int(math.sqrt(n))

    # Start tracking memory allocations
    tracemalloc.start()

    # Preprocessing
    start_preprocessing = time.time()
    blocks, sparse_table, total_comparisons = hybrid_rmq(A, block_size)
    end_preprocessing = time.time()

    # Capture memory statistics after preprocessing
    current_memory, peak_memory = tracemalloc.get_traced_memory()

    # Print preprocessing results
    print(f"Preprocessing completed in {end_preprocessing - start_preprocessing:.6f} seconds.")
    print(f"Number of comparisons during preprocessing: {total_comparisons}")
    print(f"Memory usage during preprocessing:")
    print(f"  Current memory usage: {current_memory / 1024:.2f} KB")
    print(f"  Peak memory usage: {peak_memory / 1024:.2f} KB")
    print("Time complexity of preprocessing: O(n log log n)")

    # Perform RMQ on the chosen range
    if 0 <= i <= j < n:
        start_query = time.time()
        min_value = query_hybrid_rmq(A, blocks, sparse_table, block_size, i, j)
        end_query = time.time()

        # Capture memory statistics during query
        current_memory_query, peak_memory_query = tracemalloc.get_traced_memory()

        # Print query results
        print(f"\nThe minimum value in the range [{i}, {j}] is: {min_value}")
        print(f"Query completed in {end_query - start_query:.6f} seconds.")
        print(f"Number of comparisons during query: 1")
        print(f"Memory usage during query:")
        print(f"  Current memory usage: {current_memory_query / 1024:.2f} KB")
        print(f"  Peak memory usage: {peak_memory_query / 1024:.2f} KB")
        print("Time complexity of query: O(1)")
    else:
        print("Invalid range. Please ensure 0 <= i <= j < {}.".format(n))

    # Stop memory tracking
    tracemalloc.stop()


if __name__ == "__main__":
    main()

Preprocessing completed in 0.000999 seconds.
Number of comparisons during preprocessing: 71
Memory usage during preprocessing:
  Current memory usage: 4.14 KB
  Peak memory usage: 4.45 KB
Time complexity of preprocessing: O(n log log n)

The minimum value in the range [10, 30] is: 1411
Query completed in 0.000000 seconds.
Number of comparisons during query: 1
Memory usage during query:
  Current memory usage: 5.19 KB
  Peak memory usage: 5.93 KB
Time complexity of query: O(1)


# Approach 6: Another Hybrid

In [8]:
class HybridRMQ:
    def __init__(self, array):
        self.array = array
        self.n = len(array)
        self.block_size = int(math.log2(self.n))  # Block size b = log n
        self.num_blocks = math.ceil(self.n / self.block_size)
        self.block_mins = [float('inf')] * self.num_blocks
        self.sparse_table = []
        self.block_rmq = []
        self.comparisons = 0

        # Start measuring memory usage and time for preprocessing using tracemalloc
        tracemalloc.start()
        start_time = time.time()

        self.preprocess()

        end_time = time.time()
        current_memory, peak_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        print(f"Preprocessing completed in {end_time - start_time:.6f} seconds.")
        print(f"Number of comparisons during preprocessing: {self.comparisons}")
        print(f"Current memory usage during preprocessing: {current_memory / 1024:.2f} KB")
        print(f"Peak memory usage during preprocessing: {peak_memory / 1024:.2f} KB")
        print("Time complexity of preprocessing: O(n log log n)")

    def preprocess(self):
        # 1. Precompute RMQs within each block
        for i in range(self.num_blocks):
            block_start = i * self.block_size
            block_end = min((i + 1) * self.block_size, self.n)
            block = self.array[block_start:block_end]
            self.block_rmq.append(self.precompute_rmq_block(block))

            # Compute the minimum value of each block
            self.block_mins[i] = min(block)

        # 2. Build sparse table on block minimums
        self.build_sparse_table()

    def precompute_rmq_block(self, block):
        """Precompute RMQs for a given block using a cartesian tree or other method."""
        n = len(block)
        rmq_table = [[0] * (int(math.log2(n)) + 1) for _ in range(n)]
        
        # Initialize the first column
        for i in range(n):
            rmq_table[i][0] = i
        
        # Fill the RMQ table for the block
        j = 1
        while (1 << j) <= n:
            i = 0
            while (i + (1 << j) - 1) < n:
                self.comparisons += 1  # Track comparisons
                if block[rmq_table[i][j - 1]] < block[rmq_table[i + (1 << (j - 1))][j - 1]]:
                    rmq_table[i][j] = rmq_table[i][j - 1]
                else:
                    rmq_table[i][j] = rmq_table[i + (1 << (j - 1))][j - 1]
                i += 1
            j += 1
        
        return rmq_table

    def build_sparse_table(self):
        """Build the sparse table for block minimums."""
        n = len(self.block_mins)
        k = int(math.log2(n)) + 1
        self.sparse_table = [[0] * k for _ in range(n)]
        
        # Initialize the first column of the sparse table
        for i in range(n):
            self.sparse_table[i][0] = i
        
        # Fill the sparse table
        j = 1
        while (1 << j) <= n:
            i = 0
            while (i + (1 << j) - 1) < n:
                self.comparisons += 1  # Track comparisons
                if self.block_mins[self.sparse_table[i][j - 1]] < self.block_mins[self.sparse_table[i + (1 << (j - 1))][j - 1]]:
                    self.sparse_table[i][j] = self.sparse_table[i][j - 1]
                else:
                    self.sparse_table[i][j] = self.sparse_table[i + (1 << (j - 1))][j - 1]
                i += 1
            j += 1

    def query(self, l, r):
        """Query the RMQ for the range [l, r]."""

        # Measure memory and time for querying using tracemalloc
        tracemalloc.start()
        start_time = time.time()

        self.query_comparisons = 0  # Reset comparison count for querying

        # Determine which blocks the range spans
        block_l = l // self.block_size
        block_r = r // self.block_size

        # Case 1: Query within the same block
        if block_l == block_r:
            result = self.query_rmq_block(block_l, l % self.block_size, r % self.block_size)
        else:
            # Case 2: Query over multiple blocks
            min_left = self.query_rmq_block(block_l, l % self.block_size, self.block_size - 1)
            min_right = self.query_rmq_block(block_r, 0, r % self.block_size)
            if block_r - block_l > 1:
                min_mid = self.query_sparse_table(block_l + 1, block_r - 1)
                result = min(min_left, min_mid, min_right)
            else:
                result = min(min_left, min_right)

        end_time = time.time()
        current_memory, peak_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        # Print query analysis results
        print(f"Query completed in {end_time - start_time:.6f} seconds.")
        print(f"Number of comparisons during query: {self.query_comparisons}")
        print(f"Current memory usage during query: {current_memory / 1024:.2f} KB")
        print(f"Peak memory usage during query: {peak_memory / 1024:.2f} KB")
        print("Time complexity of query: O(log log n)")

        return result

    def query_rmq_block(self, block_id, l, r):
        """Query the RMQ within a block."""
        block = self.array[block_id * self.block_size:(block_id + 1) * self.block_size]
        rmq_table = self.block_rmq[block_id]
        length = r - l + 1
        k = int(math.log2(length))
        self.query_comparisons += 1  # Track comparisons
        return min(block[rmq_table[l][k]], block[rmq_table[r - (1 << k) + 1][k]])

    def query_sparse_table(self, l, r):
        """Query the sparse table for block-level RMQ."""
        length = r - l + 1
        k = int(math.log2(length))
        self.query_comparisons += 1  # Track comparisons
        return min(self.block_mins[self.sparse_table[l][k]], self.block_mins[self.sparse_table[r - (1 << k) + 1][k]])


if main():

    rmq = HybridRMQ(A)
    
    if 0 <= i < len(A) and 0 <= j < len(A) and i <= j:
        print(f"RMQ({i}, {j}) = {rmq.query(i, j)}")
    else:
        print("Invalid indices. Please make sure 0 <= i <= j < len(array).")



if __name__ == "__main__":
    main()

Preprocessing completed in 0.000000 seconds.
Number of comparisons during preprocessing: 71
Memory usage during preprocessing:
  Current memory usage: 3.29 KB
  Peak memory usage: 3.59 KB
Time complexity of preprocessing: O(n log log n)

The minimum value in the range [10, 30] is: 1411
Query completed in 0.000000 seconds.
Number of comparisons during query: 1
Memory usage during query:
  Current memory usage: 4.40 KB
  Peak memory usage: 4.72 KB
Time complexity of query: O(1)
Preprocessing completed in 0.000000 seconds.
Number of comparisons during preprocessing: 71
Memory usage during preprocessing:
  Current memory usage: 1.90 KB
  Peak memory usage: 2.20 KB
Time complexity of preprocessing: O(n log log n)

The minimum value in the range [10, 30] is: 1411
Query completed in 0.000000 seconds.
Number of comparisons during query: 1
Memory usage during query:
  Current memory usage: 2.60 KB
  Peak memory usage: 2.81 KB
Time complexity of query: O(1)
